In [1]:
import numpy as np

# Play Tennis dataset
data = [
    ['Sunny', 'Hot', 'High', 'Weak', 'No'],
    ['Sunny', 'Hot', 'High', 'Strong', 'No'],
    ['Overcast', 'Hot', 'High', 'Weak', 'Yes'],
    ['Rain', 'Mild', 'High', 'Weak', 'Yes'],
    ['Rain', 'Cool', 'Normal', 'Weak', 'Yes'],
    ['Rain', 'Cool', 'Normal', 'Strong', 'No'],
    ['Overcast', 'Cool', 'Normal', 'Strong', 'Yes'],
    ['Sunny', 'Mild', 'High', 'Weak', 'No'],
    ['Sunny', 'Cool', 'Normal', 'Weak', 'Yes'],
    ['Rain', 'Mild', 'Normal', 'Weak', 'Yes'],
    ['Sunny', 'Mild', 'Normal', 'Strong', 'Yes'],
    ['Overcast', 'Mild', 'High', 'Strong', 'Yes'],
    ['Overcast', 'Hot', 'Normal', 'Weak', 'Yes'],
    ['Rain', 'Mild', 'High', 'Strong', 'No']
]

label_mapping = {'Yes': 1, 'No': 0}
for i in data:
    i[-1] = label_mapping[i[-1]]

# Convert categorical features to numerical values (same as before)
feature_mapping = {
    'Sunny': 0, 'Overcast': 1, 'Rain': 2,
    'Hot': 0, 'Mild': 1, 'Cool': 2,
    'High': 0, 'Normal': 1,
    'Weak': 0, 'Strong': 1
}
for instance in data:
    for i in range(len(instance) - 1):
        instance[i] = feature_mapping[instance[i]]


data = np.array(data, dtype=int)


class Node:
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, value=None):
       #The constructor is called when a new instance of the class is created
        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value # the class label assigned to the leaf node


def gini_impurity(y):   #array of class labels for a set of data points.
#np.unique(y, return_counts=True) returns two values: the unique elements in the array y and their corresponding counts.
#By using _ as the variable name, you're indicating that you're not interested in the unique elements themselves, only in the counts.
#The counts variable will store the counts of each unique element in the y array.
# using _ in this context is a way to signal that you're not using one of the values returned by the function, and you're interested only in the other value
    _, counts = np.unique(y, return_counts=True)
    #unique values in the class y and their counts
    p = counts / len(y)   #probabilities of each class label
    gini = 1.0 - np.sum(p ** 2)   #np.sum(probs ** 2) calculates the sum of squared probabilities.
    return gini


def split_dataset(X, y, feature_index, threshold):
    left_mask = X[:, feature_index] == threshold
    #creates a boolean mask (left_mask) based on a condition.
    #It checks if the values in the column specified by feature_index
    #of the array X are equal to the provided threshold
    right_mask = ~left_mask
    X_left, y_left = X[left_mask], y[left_mask]
   # uses the boolean mask left_mask to select the instances
   #from arrays X and y that satisfy the condition.
   #X_left will contain the feature values of the left subtree instances,
  # and y_left will contain their corresponding labels.
    X_right, y_right = X[right_mask], y[right_mask]
    return X_left, y_left, X_right, y_right




def find_best_split(X, y):
    num_features = X.shape[1]
    best_gini = float('inf')
    best_feature = None
    best_threshold = None

    for feature_index in range(num_features):
        unique_values = np.unique(X[:, feature_index])
        for threshold in unique_values:
            X_left, y_left, X_right, y_right = split_dataset(X, y, feature_index, threshold)
            if len(X_left) == 0 or len(X_right) == 0:
                continue

            gini_left = gini_impurity(y_left)
            gini_right = gini_impurity(y_right)
            weighted_gini = (len(X_left) / len(X)) * gini_left + (len(X_right) / len(X)) * gini_right

            if weighted_gini < best_gini:
                best_gini = weighted_gini
                best_feature = feature_index
                best_threshold = threshold

    return best_feature, best_threshold

def build_tree(X, y, depth=0, max_depth=3):
    if depth >= max_depth or len(np.unique(y)) == 1:
        leaf_value = np.argmax(np.bincount(y))
        return Node(value=leaf_value)

    best_feature, best_threshold = find_best_split(X, y)
    if best_feature is None:
        leaf_value = np.argmax(np.bincount(y))
        return Node(value=leaf_value)

    X_left, y_left, X_right, y_right = split_dataset(X, y, best_feature, best_threshold)
    left_subtree = build_tree(X_left, y_left, depth + 1, max_depth)
    right_subtree = build_tree(X_right, y_right, depth + 1, max_depth)

    return Node(feature_index=best_feature, threshold=best_threshold, left=left_subtree, right=right_subtree)



def predict_tree(node, x):
    if node.value is not None:
        return node.value

    if x[node.feature_index] == node.threshold:
        return predict_tree(node.left, x)
    else:
        return predict_tree(node.right, x)

#def print_tree(node, indent=''):
    #if node.value is not None:
        #print(f"{indent}Predict: {node.value}")
   # else:
       # print(f"{indent}Feature {node.feature_index} < {node.threshold}?")
       # print_tree(node.left, indent + '  ')
       # print_tree(node.right, indent + '  ')



# Separate features (X) and labels (y)
X = data[:, :-1]
y = data[:, -1]

# Build a decision tree
tree = build_tree(X, y, max_depth=3)

# Sample instance (same as before)
sample = np.array([1, 0, 0, 0])  # Overcast, Hot, High, Weak

# Predict using the decision tree
predicted_class = predict_tree(tree, sample)

# Print the decision tree
#print_tree(tree)
print(f"Sample Instance: {sample}")
print(f"Predicted Class: {predicted_class}")


Sample Instance: [1 0 0 0]
Predicted Class: 1
